# Welcome to the instructional evolutionary coupling notebook
# First we will load in all modules

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from biopandas.pdb import PandasPdb
import numpy as np
import statistics
import seaborn as sns

# Next we will load in all functions needed to compute the analysis

In [ ]:
# define all functions

def distance_calc(tmp_df):

    res_1 = subsubpdb.loc[(subsubpdb['chain_id'] == tmp_df.segment_i) & (subsubpdb['residue_number'] == int(tmp_df.i))]
    res_2 = subsubpdb.loc[(subsubpdb['chain_id'] == tmp_df.segment_j) & (subsubpdb['residue_number'] == int(tmp_df.j))]
    #print(res_1.as_matrix(columns=res_2.columns[11:14]))
    dist = np.linalg.norm(res_1.as_matrix(columns=res_2.columns[11:14])-res_2.as_matrix(columns=res_2.columns[11:14]))
    return dist

def fg(n):
    try:
        a = intra_df.loc[intra_df.MA.ge(n)].index[0]
        return a
    except:
        print('None')
        
def new_coupling_cutoff(tmp_df, cutoff):
    cutoff_threshold = cutoff # the distance cutoff threshold, we will be testing 20, 15, and 10
    top_scores = tmp_df.head(fg(cutoff_threshold))
    cutoff_score = top_scores['cn'].iloc[-1]
    inter_top_scores = top_scores[(top_scores['segment_i']==chain1) & (top_scores['segment_j']==chain2)]
    inter_scores = tmp_df[(tmp_df['segment_i']==chain1) & (tmp_df['segment_j']==chain2)]

    return cutoff_score, inter_top_scores, inter_scores

# The following will walk through the computation

In [ ]:
# set all variables
coev_file = '/home/cns-mccafferty/Ecoli_CoEv_modeling/output/C1CMI6_C1CMI5/couplings/C1CMI6_C1CMI5_CouplingScores.csv' # this is the location of the coupling file
pdb_file = '4I98' # this is the name of the pdb being analyzed
chain1 = 'A' # this is the first chain in the name of the coupling file
chain2 = 'C' # this is the second chain in the name of the coupling file

In [ ]:
# open coev scores file
coev = pd.read_csv(coev_file)
# open pdb as a dataframe
# extract dataframes with record_name atom and atom_name CB
ppdb = PandasPdb().fetch_pdb(pdb_file)
#ppdb = PandasPdb().read_pdb('/home/cns-mccafferty/IMP_IFTA/' + pdb_file)
#ppdb2 = PandasPdb().read_pdb('/home/cns-mccafferty/Ecoli_cellulosesecsys/P0ADJ3/' + pdbfile2)

In [ ]:
pdbdict = ppdb.df['ATOM']
subpdb = pdbdict[(pdbdict['atom_name']=='CB')]
subsubpdb = subpdb[(subpdb['chain_id']==chain1) | (subpdb['chain_id']==chain2)]

# pulls out the top 50,000 couplings
tmp_df = coev.head(50000)
tmp_df.loc[tmp_df.segment_i == 'A_1', 'segment_i'] = chain1
tmp_df.loc[tmp_df.segment_i == 'B_1', 'segment_i'] = chain2
tmp_df.loc[tmp_df.segment_j == 'A_1', 'segment_j'] = chain1
tmp_df.loc[tmp_df.segment_j == 'B_1', 'segment_j'] = chain2

# calculates distance for all pairs selected from evolutionary couplings
tmp_df['distance'] = tmp_df.apply(distance_calc, axis=1)
tmp_df = tmp_df[tmp_df.distance != 0]
print(tmp_df.head())
# calculate rolling mean for intramolecular predictions on increments of 10
intra_df = tmp_df[(tmp_df['segment_i']==tmp_df['segment_j'])]
#intra_df = intra_df[intra_df.distance != 0]
intra_df['MA'] = intra_df['distance'].rolling(10).mean()

In [ ]:
cutoff20, coev20, inter_scores = new_coupling_cutoff(tmp_df, 20)
cutoff15, coev15, inter_scores = new_coupling_cutoff(tmp_df, 15)
cutoff10, coev10, inter_scores = new_coupling_cutoff(tmp_df, 10)

In [ ]:
# saves intra- and inter-molecular pairs to csv files
path = '/home/cns-mccafferty/Ecoli_CoEv_modeling/output/C1CMI6_C1CMI5/couplings/' # change this to the correct path
coev20.to_csv(path + '/inter_' +pdb_file+chain1+chain2 + '_20cutoff.csv')
coev15.to_csv(path + '/inter_' +pdb_file+chain1+chain2 + '_15cutoff.csv')
coev10.to_csv(path + '/inter_' +pdb_file+chain1+chain2 + '_10cutoff.csv')

# Detecting intermolecular couplings between identical subunits (skip if you are not trying to do this)

In [ ]:
# calculate the 100 pair rolling mean 
intra_df['MAL'] = intra_df['distance'].rolling(100).mean().shift(-99)

In [ ]:
# calculate the 100 pair standard dev
intra_df['Std'] = intra_df['distance'].rolling(100).std().shift(-99)

In [ ]:
# calculate the 2std +- average for each point
intra_df['underline'] = intra_df['MAL'] - 2 * intra_df['Std']
intra_df['overline'] = intra_df['MAL'] + 2 * intra_df['Std']

In [ ]:
# the strict threshold uses the 10A cutoff
strict = intra_df[intra_df['cn'] > cutoff10]

In [ ]:
# intermolecular contacts inferred based on intras that are gerater than avg + 2std
inters = strict[strict['distance'] > strict['overline']]

In [ ]:
# showing the inferred inters
inters.head(20)

In [ ]:
# inferred inters are removed and new 10A cutoff is calculated and we repeated the above with new cutoff
intra_df = intra_df.drop(inters.index)
intra_df['MA'] = intra_df['distance'].rolling(10).mean()
cutoff102, coev10, inter_scores = new_coupling_cutoff(tmp_df, 10)
strict2 = intra_df[intra_df['cn'] > cutoff102]
inters2 = pd.concat([strict2[strict2['distance'] > strict2['overline']],inters])

In [ ]:
cutoff102

In [ ]:
inters2

In [ ]:
# inferred inters are removed and new 10A cutoff is calculated and we repeated the above with new cutoff
intra_df = intra_df.drop(strict2[strict2['distance'] > strict2['overline']].index)
intra_df['MA'] = intra_df['distance'].rolling(10).mean()
cutoff103, coev10, inter_scores = new_coupling_cutoff(tmp_df, 10)
strict3 = intra_df[intra_df['cn'] > cutoff103]
inters3 = pd.concat([strict3[strict3['distance'] > strict3['overline']],inters2])

In [ ]:
cutoff103

In [ ]:
inters3

In [ ]:
# inferred inters are removed and new 10A cutoff is calculated and we repeated the above with new cutoff
intra_df = intra_df.drop(strict3[strict3['distance'] > strict3['overline']].index)
intra_df['MA'] = intra_df['distance'].rolling(10).mean()
cutoff104, coev10, inter_scores = new_coupling_cutoff(tmp_df, 10)
strict4 = intra_df[intra_df['cn'] > cutoff104]
inters4 = pd.concat([strict4[strict4['distance'] > strict4['overline']],inters3])

In [ ]:
cutoff104

In [ ]:
inters4

In [ ]:
# inferred inters are removed and new 10A cutoff is calculated and we repeated the above with new cutoff
intra_df = intra_df.drop(strict4[strict4['distance'] > strict4['overline']].index)
intra_df['MA'] = intra_df['distance'].rolling(10).mean()
cutoff105, coev10, inter_scores = new_coupling_cutoff(tmp_df, 10)
strict5 = intra_df[intra_df['cn'] > cutoff105]
inters5 = pd.concat([strict5[strict5['distance'] > strict5['overline']],inters4])

In [ ]:
cutoff105

In [ ]:
# inferred inters are removed and new 10A cutoff is calculated and we repeated the above with new cutoff
intra_df = intra_df.drop(strict5[strict5['distance'] > strict5['overline']].index)
intra_df['MA'] = intra_df['distance'].rolling(10).mean()
cutoff106, coev10, inter_scores = new_coupling_cutoff(tmp_df, 10)

In [ ]:
# cutoff doesn't change so we know we can stop at the fifth iteration
cutoff106

In [ ]:
# get a list of all intra contacts that are predicted to actually be inter contacts
intratointer = inters5[inters5['segment_i'] == 'C'].drop(['distance', 'MA', 'MAL', 'Std', 'underline', 'overline'], axis=1)
intratointer['segment_i'] = intratointer['segment_i'].replace(['C'], 'B')
pdbdict = ppdb.df['ATOM']
subpdb = pdbdict[(pdbdict['atom_name']=='CB')]
subsubpdb = subpdb[(subpdb['chain_id']=='B') | (subpdb['chain_id']=='C')]
intratointer['distance'] = intratointer.apply(distance_calc, axis=1) # calculates new distances based on inters

In [ ]:
# listing all contacts with new distance
intratointer

In [ ]:
intratointer = intratointer[intratointer.distance != 0]
print(intratointer.head())

# Graphing-- this will need to be adjusted based on the data that is being plotted

In [ ]:
path = '/home/cns-mccafferty/Ecoli_CoEv_modeling/output/C1CMI6_C1CMI5/couplings/' # change this to the correct path

In [ ]:
plt.rcParams.update({'font.size': 20})
pal =sns.cubehelix_palette(4)
pal.as_hex()

In [ ]:
ax = intra_df.plot(kind='scatter', x='cn', y='distance',
                                           color='#898D6C', label='Intramolecular Contacts', figsize=(12,7.5), s=50)
inters.plot(kind='scatter', x='cn', y='distance',
                                           color='darkslategrey', label='Intramolecular Outliers', ax=ax, s=50)
# intratointer.plot(kind='scatter', x='cn', y='distance',
#                                            color='#2d1e3e', label='Intermolecular Contacts', ax=ax, s=50)
# inter_scores.plot(kind='scatter', x='cn', y='distance',
#                                          color='#edd1cb', label='Intermolecular Contacts', ax=ax, s=50)
# ax.axvline(x=cutoff20, color='black', label='20 A Cutoff')
# ax.axvline(x=cutoff15, color='black', linestyle='--', label='15 A Cutoff')
ax.axvline(x=cutoff10, color='black', linestyle=':', label='10 A Cutoff')
# ax.axvline(x=cutoff102, color='black', linestyle=':')
# ax.axvline(x=cutoff103, color='black', linestyle=':')
# ax.axvline(x=cutoff104, color='black', linestyle=':')
# ax.axvline(x=cutoff105, color='black', linestyle=':', label='10 A Cutoff')

intra_df.plot(kind='line', x='cn', y='MAL',
                                    color='black', ax=ax, label='Moving Average')

# ax.fill_between(strict5['cn'],strict5['underline'], strict5['overline'], color='grey', alpha=0.2)

# coev20.plot(kind='scatter', x='cn', y='distance',
#                                            color='#c8879e', label='Predicted 20 A Intermolecular Contacts', ax=ax, s=50)
# coev15.plot(kind='scatter', x='cn', y='distance',
#                                            color='#834c7d', label='Predicted 15 A Intermolecular Contacts', ax=ax, s=50)
# coev10.plot(kind='scatter', x='cn', y='distance',
#                                            color='#2d1e3e', label='Predicted 10 A Intermolecular Contacts', ax=ax, s=50)
plt.legend()
plt.xlim(0, 1.61) # adjust as needed
# plt.savefig(path+pdb_file+chain1+chain2+'_adjustinterall_dist.png')
plt.savefig(path+'newlegend.png')

